In [1]:
# Cell 1: 라이브러리 임포트 및 설정
import pandas as pd
import numpy as np
from est_utils import BACKBONE_CLASSES

INPUT_CSV_PATH = '04_softlabel_dataset.csv'
soft_label_cols = [f'soft_{cls}' for cls in BACKBONE_CLASSES]
epsilon = 1e-6 # 부동 소수점 비교를 위한 작은 값

In [2]:
# Cell 2: 데이터 로드
try:
    df_softlabel = pd.read_csv(INPUT_CSV_PATH)
    print(f"데이터 로드 성공: {INPUT_CSV_PATH}, {len(df_softlabel)} 행")
except FileNotFoundError:
    print(f"오류: {INPUT_CSV_PATH} 파일을 찾을 수 없습니다. 04_soft_labeling_processor.ipynb를 먼저 실행하여 파일을 생성하세요.");
    df_softlabel = pd.DataFrame() # 빈 데이터프레임으로 초기화
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    df_softlabel = pd.DataFrame()

데이터 로드 성공: 04_softlabel_dataset.csv, 8394 행


In [3]:
# Cell 3: 유효성 검사
if not df_softlabel.empty:
    print("--- 소프트 레이블 데이터 유효성 검사 시작 ---")

    # 검사 1: 소프트 레이블 합계가 1.0인지 확인
    sum_soft_labels = df_softlabel[soft_label_cols].sum(axis=1)
    deviation = (sum_soft_labels - 1.0).abs()
    if (deviation > epsilon).any():
        print(f"경고: 소프트 레이블 합계가 1.0이 아닌 행이 {deviation[deviation > epsilon].count()}개 발견되었습니다.");
        print(deviation[deviation > epsilon].head())
    else:
        print("검사 1 통과: 모든 소프트 레이블 합계가 1.0에 가깝습니다.")

    # 검사 2: 음수 값 없음 확인
    if (df_softlabel[soft_label_cols] < 0).any().any():
        print("오류: 소프트 레이블에 음수 값이 발견되었습니다.")
    else:
        print("검사 2 통과: 소프트 레이블에 음수 값이 없습니다.")

    # 검사 3: 결측값(NaN) 없음 확인
    if df_softlabel[soft_label_cols].isnull().any().any():
        print("오류: 소프트 레이블에 결측값(NaN)이 발견되었습니다.")
        print(df_softlabel[soft_label_cols].isnull().sum())
    else:
        print("검사 3 통과: 소프트 레이블에 결측값이 없습니다.")

    # 검사 4: filename 및 img_path 결측값 확인
    if df_softlabel[['filename', 'img_path']].isnull().any().any():
        print("경고: 'filename' 또는 'img_path'에 결측값이 발견되었습니다.")
        print(df_softlabel[['filename', 'img_path']].isnull().sum())
    else:
        print("검사 4 통과: 'filename' 및 'img_path'에 결측값이 없습니다.")

    # 검사 5: 'exists' 컬럼 분포 확인 (실제 파일 존재 여부)
    print("'exists' 컬럼 분포 (실제 이미지 파일 존재 여부):")
    print(df_softlabel['exists'].value_counts())
    if not df_softlabel['exists'].all():
        print("경고: 일부 이미지 파일이 실제로 존재하지 않습니다. (exists=False)")

    print("--- 소프트 레이블 데이터 유효성 검사 완료 ---")
else:
    print("데이터프레임이 비어있어 유효성 검사를 수행할 수 없습니다.")

--- 소프트 레이블 데이터 유효성 검사 시작 ---
검사 1 통과: 모든 소프트 레이블 합계가 1.0에 가깝습니다.
검사 2 통과: 소프트 레이블에 음수 값이 없습니다.
검사 3 통과: 소프트 레이블에 결측값이 없습니다.
검사 4 통과: 'filename' 및 'img_path'에 결측값이 없습니다.
'exists' 컬럼 분포 (실제 이미지 파일 존재 여부):
exists
True    8394
Name: count, dtype: int64
--- 소프트 레이블 데이터 유효성 검사 완료 ---
